# camel 使用

In [1]:

%run camel_package.ipynb

In [2]:
import os

assistant_role_name = "Python Programmer"
user_role_name = "Stock Trader"
task = "Develop a trading bot for the stock market"
word_limit = 50  # word limit for task brainstorming

In [3]:
task_specifier_sys_msg = SystemMessage(content="You can make a task more specific.")
task_specifier_prompt = """Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
Please make it more specific. Be creative and imaginative.
Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
task_specifier_template = HumanMessagePromptTemplate.from_template(
    template=task_specifier_prompt
)
llm = ChatOpenAI(temperature=0.9,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY'))

task_specify_agent = CAMELAgent(task_specifier_sys_msg, llm)
task_specifier_msg = task_specifier_template.format_messages(
    assistant_role_name=assistant_role_name,
    user_role_name=user_role_name,
    task=task,
    word_limit=word_limit,
)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Specified task: {specified_task_msg.content}")
specified_task = specified_task_msg.content

Specified task: Develop a Python-based trading bot that uses machine learning algorithms to analyze market data, predict stock price fluctuations, and execute high-frequency trades. The bot should be able to learn from past trades to make more accurate predictions and adapt to changing market conditions in real-time.


In [4]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(
    assistant_role_name, user_role_name, specified_task
)
llm2 = ChatOpenAI(temperature=0.9,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY'))
llm3 = ChatOpenAI(temperature=0.9,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY'))

assistant_agent = CAMELAgent(assistant_sys_msg, llm2)
user_agent = CAMELAgent(user_sys_msg, llm3)

# Reset agents
assistant_agent.reset()
user_agent.reset()

# Initialize chats
user_msg = HumanMessage(
    content=(
        f"{user_sys_msg.content}. "
        "Now start to give me introductions one by one. "
        "Only reply with Instruction and Input."
    )
)

assistant_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
assistant_msg = assistant_agent.step(user_msg)

## Start role-playing session to solve the task!

In [9]:
def run(user_agent,assistant_agent,assistant_msg,chat_turn_limit=30,n=0):

    while n < chat_turn_limit:
        n += 1
        user_ai_msg = user_agent.step(assistant_msg)
        user_msg = HumanMessage(content=user_ai_msg.content)
        print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")

        assistant_ai_msg = assistant_agent.step(user_msg)
        assistant_msg = HumanMessage(content=assistant_ai_msg.content)
        print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
        if "<CAMEL_TASK_DONE>" in user_msg.content:
            break

In [ ]:
run(user_agent,assistant_agent,assistant_msg)